In [ ]:
import bs4 as bs
import urllib.request
import re
import nltk
import numpy as np
import random
import string


In [ ]:
nltk.download('punkt')

In [ ]:
!pip install spacy==2.2.3


In [ ]:
import spacy
spacy.__version__

In [ ]:
nltk.__version__

#Spacy where we select english languaje

In [ ]:
!python3 -m spacy download es


###Upload & preprocessing DB data

In [ ]:
data = urllib.request.urlopen('hhttps://www.hiberus.com/crecemos-contigo/buscador-palabras-clave-las-mejores-herramientas-para-busqueda-de-keywords/')

In [ ]:
data = data.read()
data

In [ ]:
data_html = bs.BeautifulSoup(data, 'lxml')
data_html

In [ ]:
paragraphs = data_html.find_all('p')


In [ ]:
len(paragraphs)

In [ ]:
paragraphs[0].text

In [ ]:
content = ''
for p in paragraphs:
  content += p.text

In [ ]:
content

##Preprocess

In [ ]:
content = content.lower()
content

Convert in array of sentences : ["content"]

In [ ]:
list_sentences = nltk.sent_tokenize(content)
list_sentences

In [ ]:
len(list_sentences)

In [ ]:
list_sentences[0]

#We import NLP in English from spacy

In [ ]:
import spacy
from spacy.lang.es.examples import sentences

nlp = spacy.load('es')
nlp

In [ ]:
stop_words = spacy.lang.es.stop_words.STOP_WORDS

In [ ]:
print(stop_words)

In [ ]:
len(stop_words)

# Characters that we gonna eliminate 

In [ ]:
string.punctuation

#Function preprocess

Line 3: We use regex to remove URL's & replace it by a blank space and after, we eliminate it in line 5

Line 7: We work in lemmatization(normalize a standar word)

Line 12: We eliminate stopwords and punctiation


In [ ]:
def preprocess(text):
#URLs bye
  text=re.sub(r"https?://[A-Za-z0-9./]+",'',text) 
  #Blank spaces
  text = re.sub(r" +"," ", text)

  document = nlp(text)
  list = []
  for token in document:
    list.append(token.lemma_)
  
  list = [word for word in list if word not in stop_words and word not in string.punctuation]
  list = ' '.join([str(element) for element in list if not element.isdigit()])

  return list

In [ ]:
text_test = 'https://www.hiberus.com/crecemos-contigo/buscador-palabras-clave-las-mejores-herramientas-para-busqueda-de-keywords/' + list_sentences[0]

In [ ]:
text_test

In [ ]:
result = preprocess(text_test)
result

In [ ]:
list_sentences_preprocess = []
for i in range(len(list_sentences)):
  list_sentences_preprocess.append(preprocess(list_sentences[i]))
  

#Wellcome phrases

In [ ]:
input_greetings = ('hi','hello','hey','ohana','ohayo','whats up?','que onda')
output_greetings = ('wellcome','good to see you','hello!','hi!', 'how are you?')

In [ ]:
def reply_greeting(text):
  for word in text.split():
    if word.lower() in input_greetings:
      return random.choice(output_greetings)

In [ ]:
reply_greeting('Ohayo bro')


#TF-IDF(Term frequency-Inverse Document Frequency)
TF = Total of times word apear / Total number terms
IDF = log(N/n)

In [ ]:
 pip install scikit-learn

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
phrases_test = list_sentences_preprocess[:3]
phrases_test

In [ ]:
phrases_test.append(phrases_test[0])
phrases_test

In [ ]:
vectorizator = TfidfVectorizer()
words_vectorized = vectorizator.fit_transform(phrases_test)

In [ ]:
type(words_vectorized)

In [ ]:
words_vectorized

In [ ]:
print(vectorizator.get_feature_names())

In [ ]:
len(vectorizator.get_feature_names())

In [ ]:
print(vectorizator.vocabulary_)

In [ ]:
print(vectorizator.idf_)

In [ ]:
words_vectorized.todense()

In [ ]:
words_vectorized.todense().shape

Document = Vector
Cosine = Similarity between two documents(two vectors)

Cosine Similarity (d1, d2) =  Dot product(d1, d2) / ||d1|| * ||d2||

---
Dot product (d1,d2) = d1[0] * d2[0] + d1[1] * d2[1] * … * d1[n] * d2[n]
||d1|| = square root(d1[0]2 + d1[1]2 + ... + d1[n]2)
||d2|| = square root(d2[0]2 + d2[1]2 + ... + d2[n]2)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
words_vectorized[0].todense()

In [ ]:
cosine_similarity(words_vectorized[0], words_vectorized[1])

In [ ]:
cosine_similarity(words_vectorized[0], words_vectorized[1])

In [ ]:
similarity = cosine_similarity(words_vectorized[0], words_vectorized)
similarity

In [ ]:
similarity.argsort()

In [ ]:
i = similarity.argsort()[0][-2]
i

In [ ]:
i = i.flatten()
i

#Reply to the user

In [ ]:
def answer(user_text):
  global answer_chatbot
  answer_chatbot = ''
  list_sentences_preprocess.append(user_text)

  vectorizator = TfidfVectorizer()
  words_vectorized = vectorizator.fit_transform(list_sentences_preprocess)

  similarity = cosine_similarity(words_vectorized[-1], words_vectorized)
  index_sentence = similarity.argsort()[0][-2]
  similar_vector = similarity.flatten()
  similar_vector.sort()
  vector_found = similar_vector[-2]

  if(vector_found == 0):
    answer_chatbot = answer_chatbot + 'Sorry, I did not understand, Could you write it in another way?'
    return answer_chatbot
  else:
    answer_chatbot = answer_chatbot + list_sentences[index_sentence]
    return answer_chatbot


#Let's Talk!


In [ ]:
continue_talking = True
print('Hi! You can tell me your questions about internal process')
while(continue_talking == True):
  user_text = input()
  user_text = user_text.lower()
  if(user_text != 'exit'):
    if(reply_greeting(user_text) != None):
      print('Chatbot:' + reply_greeting(user_text))
    else:
      print('Chatbot: ')
      print(answer(preprocess(user_text)))
      list_sentences_preprocess.remove(preprocess(user_text))

  else:
    continue_talking = False
    print('Bye bye!')

Hi! You can tell me your questions about internal process
Chatbot: 
Sorry, I did not understand, Could you write it in another way?
Chatbot:how are you?
Chatbot: 
Sorry, I did not understand, Could you write it in another way?
